<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/fine-tuning-experiments/askpatient/askApatient_overlap_90_cadec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os

In [0]:
base_dir = 'medical-concept-normalization'

In [0]:
os.mkdir(base_dir)

In [0]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [5]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   287  100   287    0     0    611      0 --:--:-- --:--:-- --:--:--   611
bash: line 1: syntax error near unexpected token `newline'
bash: line 1: `<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">'


In [6]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [0]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [0]:
path = Path('medical-concept-normalization/data_collection')

In [0]:
# training data set

train_csv = path/'AskAPatient.fold-0.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [12]:
# validating data set

valid = pd.read_csv(path/"AskAPatient.fold-0.validation.csv",header=None)
valid.head()

,0,1
0,0,fear
1,2,heel pain
2,4,purple sploches around waist
3,7,goiter
4,8,ARTHRITIS


In [13]:
# testing data set

test = pd.read_csv(path/"AskAPatient_overlap_90.csv",header=None)
test.head()

,0,1
0,0,'scared' feeling
1,0,fear
2,0,scared to death
3,2,heel pain
4,2,heel pain


In [0]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'fine_tuning_language_model_cadec_data.csv')

In [0]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [0]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [0]:
data_clas.add_test(test,label=0)


In [18]:
data_lm.save()
data_clas.save()

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type CrossEntropyLoss. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [0]:
data_lm.train_ds.vocab.itos

In [0]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [0]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [22]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.847058,4.199748,0.216773,03:33


In [23]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.963303,3.722538,0.291677,03:32


In [24]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.707239,3.690769,0.297895,03:32


In [25]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.662228,3.668235,0.300989,03:32


In [0]:
learn.save_encoder('AskAPatient_overlap_90_first')

In [27]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('AskAPatient_overlap_90_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.948893,3.480983,0.424852,01:07


In [28]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.947638,2.914783,0.524260,01:25


In [29]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.279290,2.598511,0.578698,03:13


In [30]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.055674,2.273353,0.623669,03:24
1,1.779035,2.192333,0.639053,03:07


In [31]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.735173,1.948857,0.671006,03:34
1,1.421479,1.876408,0.684024,03:14


In [32]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.353964,1.798844,0.695858,03:11
1,1.178435,1.718409,0.713609,03:08


In [33]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.131983,1.671735,0.720710,03:20
1,1.009063,1.616799,0.732544,03:32


In [34]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.030649,1.630031,0.736095,03:08
1,0.915509,1.599139,0.733728,03:25


In [35]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.986750,1.544275,0.745562,03:23
1,0.873906,1.529380,0.746746,03:35
2,0.772711,1.437171,0.753846,03:12
3,0.702322,1.459002,0.763314,03:30


In [36]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.710717,1.480307,0.755030,03:24
1,0.734027,1.477621,0.751479,03:28
2,0.666636,1.429826,0.763314,03:07
3,0.619262,1.431201,0.765680,03:17


In [37]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.617076,1.430377,0.760947,03:20
1,0.654654,1.477398,0.759763,03:14
2,0.632480,1.408372,0.760947,03:07
3,0.573336,1.417867,0.772781,03:18


In [38]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.591603,1.434365,0.776331,03:11
1,0.604316,1.444488,0.766864,03:24
2,0.566567,1.453680,0.776331,03:17
3,0.516858,1.422846,0.777515,03:17


In [39]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.505363,1.446331,0.775148,03:36
1,0.559183,1.501790,0.771598,03:31
2,0.555461,1.495515,0.772781,03:05
3,0.489536,1.483558,0.777515,03:36


In [40]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.498239,1.508479,0.766864,03:33
1,0.565933,1.468739,0.772781,03:14
2,0.506383,1.474532,0.778698,03:32
3,0.463974,1.445826,0.785799,03:31


In [41]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.504702,1.533494,0.775148,03:13
1,0.549281,1.616755,0.770414,03:11
2,0.479581,1.491886,0.779882,03:27
3,0.444420,1.511668,0.777515,03:17


In [42]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.468642,1.596841,0.766864,03:21
1,0.544090,1.583641,0.779882,03:18
2,0.486787,1.531955,0.779882,03:33
3,0.457474,1.583735,0.776331,03:33


In [43]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.482608,1.499377,0.777515,03:34
1,0.475562,1.526037,0.779882,03:13
2,0.468024,1.630146,0.782248,03:11
3,0.491525,1.538425,0.786982,03:20


In [44]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.433363,1.516215,0.783432,03:25
1,0.497221,1.612204,0.775148,03:35
2,0.476174,1.557602,0.783432,03:37
3,0.444097,1.627196,0.781065,03:37


In [45]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.447136,1.595950,0.779882,03:26
1,0.458518,1.602528,0.783432,03:35
2,0.472537,1.586587,0.789349,03:17
3,0.424117,1.609625,0.788166,03:37


In [46]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.440712,1.530509,0.785799,03:22
1,0.477347,1.560635,0.778698,03:20
2,0.476962,1.687769,0.779882,03:11
3,0.423741,1.566664,0.785799,03:33


In [0]:
# save the best model

learn.save_encoder('AskAPatient_overlap_90')

# Part three: Predict on the test dataset

In [0]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [49]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

6101
6101
[0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 4, 824, 334, 5, 5, 964, 6, 6, 495, 495, 495, 495, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 495, 495, 495, 8, 8, 8, 472, 348, 10, 12, 12, 4, 15, 16, 16, 16, 16, 16, 359, 359, 1035, 788, 317, 20, 21, 21, 964, 22, 22, 23, 25, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 427, 27, 29, 29, 29, 29, 30, 30, 30, 30, 30, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 32, 32, 32, 866, 34, 34, 34, 34, 34, 34, 877, 34, 34, 928, 1031, 37, 704, 135, 135, 289, 37, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 37, 37, 37, 37, 37, 37, 37, 37, 37, 289, 289, 37, 289, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 651, 37, 37, 48, 218, 39, 39, 528, 41, 515, 41, 41, 42, 42, 42, 44, 44, 44, 45, 46, 964, 48, 48, 1035, 49, 1032, 1032, 325, 1031, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 325, 49, 49, 49, 49, 49, 4

In [50]:
correct_num = 0

for i in range(0, len(predict_list)):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

4409
0.7226684150139322
